# 03 — Depth → Point Cloud (Geometry Explainer)

This notebook explains and demonstrates how the project converts a depth map into a colored point cloud.

Key idea: given a depth map **D(u,v)** and camera intrinsics **fx, fy**, we compute:

- X = (u - cx) * D / fx
- Y = (v - cy) * D / fy
- Z = D

Then we attach the RGB color from the input image.

**Note:** MiDaS produces *relative* depth. The point cloud is therefore geometrically meaningful up to a scale/shift.

In [ ]:
# --- Standard imports ---
import os
from pathlib import Path
import json
import numpy as np

# --- Visualization ---
import matplotlib.pyplot as plt

# Make plots bigger by default (no fixed colors here)
plt.rcParams["figure.figsize"] = (10, 6)

# --- Repo root detection (works when notebook is in ./notebooks) ---
REPO_ROOT = Path.cwd()
if (REPO_ROOT / "src").exists() and (REPO_ROOT / "configs").exists():
    pass
elif (REPO_ROOT.parent / "src").exists() and (REPO_ROOT.parent / "configs").exists():
    REPO_ROOT = REPO_ROOT.parent
else:
    raise RuntimeError("Could not locate repo root. Open this notebook from inside the repo folder.")

print("Repo root:", REPO_ROOT)

# Ensure src is importable
import sys
SRC_DIR = REPO_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Imports from the project
from human3d.utils.config import load_config
from human3d.pipeline import Human3DPipeline

## 1) Load an existing run directory

Set this to a folder inside `outputs/` that already contains `depth.npy` and `pointcloud.ply`.

If you don’t have one, run notebook 01 or 02 first.

In [ ]:
# Change this to an actual run directory you created.
RUN_DIR = REPO_ROOT / "outputs"

# Auto-pick most recent run folder if possible
runs = sorted([p for p in RUN_DIR.glob("run_*") if p.is_dir()], key=lambda p: p.stat().st_mtime, reverse=True)
assert runs, f"No runs found in {RUN_DIR}. Run the pipeline first."
out_dir = runs[0]
print("Using run:", out_dir)

## 2) Inspect depth array

In [ ]:
depth = np.load(out_dir / "depth.npy")
print("depth shape:", depth.shape, "min/max:", float(depth.min()), float(depth.max()))
plt.figure()
plt.imshow(depth)
plt.axis("off")
plt.title("Raw depth array (relative depth)")
plt.show()

## 3) Reconstruct point cloud again (from depth)

In [ ]:
import cv2
from human3d.reconstruct.pointcloud import depth_to_pointcloud, save_ply

# Load original input image if summary has it
summary = json.loads((out_dir / "summary.json").read_text(encoding="utf-8"))
img_path = Path(summary["input"])
if not img_path.is_absolute():
    img_path = REPO_ROOT / img_path

bgr = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
assert bgr is not None, f"Could not read image: {img_path}"

# Intrinsics from config
cfg = load_config(str(REPO_ROOT / "configs" / "pipeline.yaml"))
pcfg = cfg.get("reconstruction", {}).get("pointcloud", {})
fx = float(pcfg.get("fx", 1000.0))
fy = float(pcfg.get("fy", 1000.0))

# Optional mask
mask_path = out_dir / "seg_mask.png"
mask01 = None
if mask_path.exists():
    m = cv2.imread(str(mask_path), cv2.IMREAD_GRAYSCALE)
    mask01 = (m > 0).astype(np.uint8)

depth_used = depth.copy()
if mask01 is not None:
    depth_used[mask01 == 0] = 0.0

pcd = depth_to_pointcloud(depth_used, bgr, fx=fx, fy=fy)
recon_path = out_dir / "pointcloud_recomputed.ply"
save_ply(str(recon_path), pcd)

print("Saved recomputed point cloud:", recon_path)

## 4) View point cloud (optional)

In [ ]:
try:
    import open3d as o3d
    pcd = o3d.io.read_point_cloud(str(out_dir / "pointcloud_recomputed.ply"))
    print(pcd)
    o3d.visualization.draw_geometries([pcd])
except Exception as e:
    print("Open3D preview failed:", repr(e))
    print("Open the PLY file in CloudCompare or MeshLab instead.")